## Semester 1 Project Submission

Please fill out:
* Student name: 
* Instructor name: 


In [1]:
# Your code here - remember to use markdown cells for comments as well!

In [2]:
import sqlite3
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from scipy import stats

conn = sqlite3.connect('zippedData/im.db')

# Dylan Orndorf-Ronk

# Alex Bonczkiewicz

# Alberto Torres

# Travis McCloughan

# Jaemin Lee

In [3]:
q = """
SELECT name 
FROM sqlite_master 
WHERE type='table';
"""
pd.read_sql(q, conn)

,name
0,movie_basics
1,directors
2,known_for
3,movie_akas
4,movie_ratings
5,persons
6,principals
7,writers


In [5]:
movie_basics = pd.read_sql("""
SELECT * 
FROM movie_basics;
""",conn)
movie_basics

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama"
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy"
...,...,...,...,...,...,...
146139,tt9916538,Kuambil Lagi Hatiku,Kuambil Lagi Hatiku,2019,123.0,Drama
146140,tt9916622,Rodolpho Teóphilo - O Legado de um Pioneiro,Rodolpho Teóphilo - O Legado de um Pioneiro,2015,NaN,Documentary
146141,tt9916706,Dankyavar Danka,Dankyavar Danka,2013,NaN,Comedy
146142,tt9916730,6 Gunn,6 Gunn,2017,116.0,None


In [6]:
movie_ratings = pd.read_sql("""
SELECT * 
FROM movie_ratings;
""",conn)
movie_ratings

,movie_id,averagerating,numvotes
0,tt10356526,8.3,31
1,tt10384606,8.9,559
2,tt1042974,6.4,20
3,tt1043726,4.2,50352
4,tt1060240,6.5,21
...,...,...,...
73851,tt9805820,8.1,25
73852,tt9844256,7.5,24
73853,tt9851050,4.7,14
73854,tt9886934,7.0,5


In [23]:
movies_df = pd.read_sql("""
SELECT *
FROM movie_akas
INNER JOIN
    movie_basics
ON  movie_akas.movie_id = movie_basics.movie_id
LEFT JOIN
    movie_ratings
ON  movie_akas.movie_id = movie_ratings.movie_id
""",conn)
movies_df

,movie_id,ordering,title,region,language,types,attributes,is_original_title,movie_id,primary_title,original_title,start_year,runtime_minutes,genres,movie_id,averagerating,numvotes
0,tt0369610,10,Джурасик свят,BG,bg,None,None,0.0,tt0369610,Jurassic World,Jurassic World,2015,124.0,"Action,Adventure,Sci-Fi",tt0369610,7.0,539338.0
1,tt0369610,11,Jurashikku warudo,JP,None,imdbDisplay,None,0.0,tt0369610,Jurassic World,Jurassic World,2015,124.0,"Action,Adventure,Sci-Fi",tt0369610,7.0,539338.0
2,tt0369610,12,Jurassic World: O Mundo dos Dinossauros,BR,None,imdbDisplay,None,0.0,tt0369610,Jurassic World,Jurassic World,2015,124.0,"Action,Adventure,Sci-Fi",tt0369610,7.0,539338.0
3,tt0369610,13,O Mundo dos Dinossauros,BR,None,None,short title,0.0,tt0369610,Jurassic World,Jurassic World,2015,124.0,"Action,Adventure,Sci-Fi",tt0369610,7.0,539338.0
4,tt0369610,14,Jurassic World,FR,None,imdbDisplay,None,0.0,tt0369610,Jurassic World,Jurassic World,2015,124.0,"Action,Adventure,Sci-Fi",tt0369610,7.0,539338.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
331698,tt9827784,2,Sayonara kuchibiru,None,None,original,None,1.0,tt9827784,Farewell Song,Sayonara kuchibiru,2019,116.0,"Music,Romance",tt9827784,7.7,6.0
331699,tt9827784,3,Farewell Song,XWW,en,imdbDisplay,None,0.0,tt9827784,Farewell Song,Sayonara kuchibiru,2019,116.0,"Music,Romance",tt9827784,7.7,6.0
331700,tt9880178,1,La atención,None,None,original,None,1.0,tt9880178,The Attention,La atención,2019,61.0,Documentary,None,NaN,NaN
331701,tt9880178,2,La atención,ES,None,None,None,0.0,tt9880178,The Attention,La atención,2019,61.0,Documentary,None,NaN,NaN


In [39]:
movies_genre = pd.read_sql("""
SELECT genres, sum(numvotes) as num_votes, AVG(averagerating) as avg_rating
FROM movie_basics
INNER JOIN movie_ratings
    ON movie_basics.movie_id = movie_ratings.movie_id
GROUP BY genres
ORDER BY num_votes DESC, avg_rating DESC
LIMIT 20
""",conn)
movies_genre

,genres,num_votes,avg_rating
0,"Action,Adventure,Sci-Fi",23023248,5.655906
1,"Action,Adventure,Fantasy",9658883,5.371845
2,"Adventure,Animation,Comedy",8687435,5.936555
3,Drama,8395521,6.494265
4,"Comedy,Drama,Romance",7665463,6.292467
5,"Action,Adventure,Comedy",7256686,5.554032
6,Comedy,6832037,5.777998
7,"Comedy,Drama",6462839,6.364119
8,"Action,Crime,Drama",5563553,5.989146
9,"Drama,Romance",5542760,6.294305


In [47]:
#movies_df.groupby(by="region").sum().reset_index()

movies_df.sort_values(['numvotes','averagerating'], ascending=False).groupby(by="region")

In [21]:
persons = pd.read_sql("""
SELECT * 
FROM PERSONS;
""",conn)
persons

,person_id,primary_name,birth_year,death_year,primary_profession
0,nm0061671,Mary Ellen Bauder,NaN,NaN,"miscellaneous,production_manager,producer"
1,nm0061865,Joseph Bauer,NaN,NaN,"composer,music_department,sound_department"
2,nm0062070,Bruce Baum,NaN,NaN,"miscellaneous,actor,writer"
3,nm0062195,Axel Baumann,NaN,NaN,"camera_department,cinematographer,art_department"
4,nm0062798,Pete Baxter,NaN,NaN,"production_designer,art_department,set_decorator"
...,...,...,...,...,...
606643,nm9990381,Susan Grobes,NaN,NaN,actress
606644,nm9990690,Joo Yeon So,NaN,NaN,actress
606645,nm9991320,Madeline Smith,NaN,NaN,actress
606646,nm9991786,Michelle Modigliani,NaN,NaN,producer


In [7]:
movie_akas = pd.read_sql("""
SELECT * 
FROM movie_akas;
""",conn)
movie_akas

,movie_id,ordering,title,region,language,types,attributes,is_original_title
0,tt0369610,10,Джурасик свят,BG,bg,None,None,0.0
1,tt0369610,11,Jurashikku warudo,JP,None,imdbDisplay,None,0.0
2,tt0369610,12,Jurassic World: O Mundo dos Dinossauros,BR,None,imdbDisplay,None,0.0
3,tt0369610,13,O Mundo dos Dinossauros,BR,None,None,short title,0.0
4,tt0369610,14,Jurassic World,FR,None,imdbDisplay,None,0.0
...,...,...,...,...,...,...,...,...
331698,tt9827784,2,Sayonara kuchibiru,None,None,original,None,1.0
331699,tt9827784,3,Farewell Song,XWW,en,imdbDisplay,None,0.0
331700,tt9880178,1,La atención,None,None,original,None,1.0
331701,tt9880178,2,La atención,ES,None,None,None,0.0


In [8]:
directors = pd.read_sql("""
SELECT * 
FROM directors;
""",conn)
directors.head()

,movie_id,person_id
0,tt0285252,nm0899854
1,tt0462036,nm1940585
2,tt0835418,nm0151540
3,tt0835418,nm0151540
4,tt0878654,nm0089502


In [19]:
principals = pd.read_sql("""
SELECT * 
FROM principals;
""",conn)
principals

,movie_id,ordering,person_id,category,job,characters
0,tt0111414,1,nm0246005,actor,None,"[""The Man""]"
1,tt0111414,2,nm0398271,director,None,None
2,tt0111414,3,nm3739909,producer,producer,None
3,tt0323808,10,nm0059247,editor,None,None
4,tt0323808,1,nm3579312,actress,None,"[""Beth Boothby""]"
...,...,...,...,...,...,...
1028181,tt9692684,1,nm0186469,actor,None,"[""Ebenezer Scrooge""]"
1028182,tt9692684,2,nm4929530,self,None,"[""Herself"",""Regan""]"
1028183,tt9692684,3,nm10441594,director,None,None
1028184,tt9692684,4,nm6009913,writer,writer,None


In [22]:
known_for = pd.read_sql("""
SELECT * 
FROM known_for;
""",conn)
known_for['movie_id'].value_counts()

tt0806910    633
tt0441074    478
tt0203259    470
tt0486531    462
tt0452046    437
            ... 
tt5481398      1
tt2913514      1
tt2328593      1
tt7520484      1
tt3000214      1
Name: movie_id, Length: 514781, dtype: int64